In [2]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29413 sha256=8b148465742d3a63660f57af2413eff9f77ac755420ea75e421143a9a736cf8a
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [81]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import yfinance as yf
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [82]:
stocks = ["KO"]
start_date = "2000-01-01"
end_date = "2023-12-01"
stock_data_list = []

# Download S&P 500 data
sp500 = yf.download('^GSPC', start=start_date, end=end_date)
sp500 = sp500[['Close']].rename(columns={'Close': 'SP500_Close'})
for stock in stocks:
    df = yf.download(stock, start=start_date, end=end_date)
    df['Stock'] = stock
    # Merge the S&P 500 close data with the stock data on the date index
    df = df.merge(sp500, left_index=True, right_index=True, how='left')

    # Calculate the correlation of the stock's close with the S&P 500 close
    df['SP500_Correlation'] = df['Close'].corr(df['SP500_Close'])

    stock_data_list.append(df)
stock_data = pd.concat(stock_data_list)
stock_data.reset_index(inplace=True)

print(stock_data.head(-1))

# After the loop, you can drop the NaN values from the correlation calculation if needed
stock_data.dropna(subset=['SP500_Correlation'], inplace=True)
stock_data.reset_index(drop=True, inplace=True)

# checking df
null_values = stock_data.isnull().sum()
print("Null values in each column:\n", null_values)

stock_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
           Date       Open       High        Low      Close  Adj Close  \
0    2000-01-03  29.000000  29.000000  27.625000  28.187500  14.549594   
1    2000-01-04  28.187500  28.406250  27.812500  28.218750  14.565716   
2    2000-01-05  28.218750  28.718750  28.031250  28.468750  14.694760   
3    2000-01-06  28.468750  28.843750  28.281250  28.500000  14.710901   
4    2000-01-07  28.937500  30.375000  28.937500  30.375000  15.678721   
...         ...        ...        ...        ...        ...        ...   
6011 2023-11-22  58.259998  58.540001  58.130001  58.419998  57.958496   
6012 2023-11-24  58.459999  58.750000  58.340000  58.570000  58.107315   
6013 2023-11-27  58.540001  58.689999  58.270000  58.459999  57.998184   
6014 2023-11-28  58.400002  58.830002  58.360001  58.580002  58.117237   
6015 2023-11-29  58.580002  58.669998  58.099998

,Date,Open,High,Low,Close,Adj Close,Volume,Stock,SP500_Close,SP500_Correlation
0,2000-01-03,29.000000,29.000000,27.625000,28.187500,14.549594,10997000,KO,1455.219971,0.925094
1,2000-01-04,28.187500,28.406250,27.812500,28.218750,14.565716,7308000,KO,1399.420044,0.925094
2,2000-01-05,28.218750,28.718750,28.031250,28.468750,14.694760,9457400,KO,1402.109985,0.925094
3,2000-01-06,28.468750,28.843750,28.281250,28.500000,14.710901,7129200,KO,1403.449951,0.925094
4,2000-01-07,28.937500,30.375000,28.937500,30.375000,15.678721,11474000,KO,1441.469971,0.925094
...,...,...,...,...,...,...,...,...,...,...
6012,2023-11-24,58.459999,58.750000,58.340000,58.570000,58.107315,4816000,KO,4559.339844,0.925094
6013,2023-11-27,58.540001,58.689999,58.270000,58.459999,57.998184,16246500,KO,4550.430176,0.925094
6014,2023-11-28,58.400002,58.830002,58.360001,58.580002,58.117237,13739600,KO,4554.890137,0.925094
6015,2023-11-29,58.580002,58.669998,58.099998,58.230000,57.770000,11263600,KO,4550.580078,0.925094


In [83]:
# Convert 'Date' column to datetime
stock_data['Date'] = pd.to_datetime(stock_data['Date'])


In [84]:
# Time Series Decomposition of KO's Close price
decomposition = seasonal_decompose(stock_data['Close'], model='additive', period=365)  # Adjust period as needed
stock_data['Trend'] = decomposition.trend
stock_data['Seasonality'] = decomposition.seasonal
stock_data['Residual'] = decomposition.resid

In [85]:
# Handling any NaN values
stock_data.fillna(method='bfill', inplace=True)
stock_data.fillna(method='ffill', inplace=True)

In [86]:
# Selecting Features and Target for the model
features = ['SP500_Close', 'SP500_Correlation','Trend', 'Seasonality', 'Residual']
X = stock_data[features]
y = stock_data['Close']  # Target variable

In [87]:
# Calculate EMA of Close price for 'KO'
ema_span = 26  # This is the span for EMA calculation, adjust as needed
stock_data['EMA'] = stock_data['Close'].ewm(span=ema_span, adjust=False).mean()

# Adding EMA to the features
features.append('EMA')


In [88]:
# Splitting Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)

## XGB

In [99]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import matplotlib.pyplot as plt

xgb_ensemble_predictions = pd.DataFrame()

# Train XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

     # Combine predictions using a simple average
ensemble_predictions_stock = pd.DataFrame({
        'Date': stock_data.loc[X_test.index, 'Date'],
        'SP500_Close' : stock_data.loc[X_test.index, 'SP500_Close'],
        'SP500_Correlation' : stock_data.loc[X_test.index, 'SP500_Correlation'],
        'Trend': stock_data.loc[X_test.index, 'Trend'],
        'Seasonality': stock_data.loc[X_test.index, 'Seasonality'],
        'Residual': stock_data.loc[X_test.index, 'Residual'],
        'Actual': y_test,
        'Stock': stock,
        'EMA': stock_data.loc[X_test.index, 'EMA'],
        'XGB': xgb_predictions,
        'Ensemble': (stock_data.loc[X_test.index, 'EMA'] + xgb_predictions) / 2
    })
# Append predictions for the current stock to the overall ensemble predictions
xgb_ensemble_predictions = pd.concat([xgb_ensemble_predictions, ensemble_predictions_stock])

# Evaluate the ensemble model
mse_xgb = mean_squared_error(xgb_ensemble_predictions['Actual'], xgb_ensemble_predictions['Ensemble'])
r2_xgb = r2_score(xgb_ensemble_predictions['Actual'], xgb_ensemble_predictions['Ensemble'])
print(f'Mean Squared Error (Ensemble): {mse_xgb}')
print(f'R-squared (Ensemble): {r2_xgb}')


Mean Squared Error (Ensemble): 0.4165465929524462
R-squared (Ensemble): 0.9974446694326564


## SVM

In [91]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

# Initialize an empty DataFrame to store predictions
ensemble_predictions_svm = pd.DataFrame()


# Train Support Vector Machine (SVM) regressor model
svm_model = SVR()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

# Combine predictions using a simple average
ensemble_predictions_stock_svm = pd.DataFrame({
        'Date': stock_data.loc[X_test.index, 'Date'],
        'SP500_Close' : stock_data.loc[X_test.index, 'SP500_Close'],
        'SP500_Correlation' : stock_data.loc[X_test.index, 'SP500_Correlation'],
        'Trend': stock_data.loc[X_test.index, 'Trend'],
        'Seasonality': stock_data.loc[X_test.index, 'Seasonality'],
        'Residual': stock_data.loc[X_test.index, 'Residual'],
        'Actual': y_test,
        'Stock': stock,
        'EMA': stock_data.loc[X_test.index, 'EMA'],
        'SVM': svm_predictions,
        'Ensemble': (stock_data.loc[X_test.index, 'EMA'] + svm_predictions) / 2
    })

    # Append predictions for the current stock to the overall ensemble predictions
ensemble_predictions_svm = pd.concat([ensemble_predictions_svm, ensemble_predictions_stock_svm])

# Evaluate the ensemble model
mse_svm = mean_squared_error(ensemble_predictions_svm['Actual'], ensemble_predictions_svm['Ensemble'])
r2_svm = r2_score(ensemble_predictions_svm['Actual'], ensemble_predictions_svm['Ensemble'])
print(f'Mean Squared Error (Ensemble - SVM): {mse_svm}')
print(f'R-squared (Ensemble - SVM): {r2_svm}')


Mean Squared Error (Ensemble - SVM): 4.8408671950180375
R-squared (Ensemble - SVM): 0.9703034039284713


In [100]:
# Display results
print(f'Model: Ensemble + SVM')
print(f'Mean Squared Error: {mse_svm:.4f}')
print(f'R-squared: {r2_svm:.4f}')
print('\n' + '-'*40 + '\n')  # Separator

print(f'Model: XGBoost Ensemble')
print(f'Mean Squared Error: {mse_xgb:.4f}')
print(f'R-squared: {r2_xgb:.4f}')

Model: Ensemble + SVM
Mean Squared Error: 4.8409
R-squared: 0.9703

----------------------------------------

Model: XGBoost Ensemble
Mean Squared Error: 0.4165
R-squared: 0.9974
